# complib.base_component

> TODO fill in description

In [ ]:
#| default_exp complib.exec_component

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide®
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from abc import abstractmethod

import fbdev
from fbdev.comp.packet import Packet
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection
from fbdev.comp.base_component import BaseComponent

In [ ]:
#|hide
show_doc(fbdev.complib.ExecComponent)

---

### ExecComponent

>      ExecComponent ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class ExecComponent(BaseComponent):
    execute_after_start = True
    
    port_specs = PortSpecCollection(
        PortSpec(PortType.SIGNAL, 'execute')
    )
    
    def __init__(self):
        super().__init__()
        self._main_task: asyncio.Task = None
    
    async def _post_start(self):
        self._task_manager.create_task(self._pre_execute())
        if self.execute_after_start:
            self._task_manager.create_task(
                self.put_packet((PortType.SIGNAL, 'execute'), Packet.get_empty())
            )
        self._main_task = self._task_manager.create_task(self._pre_execute())
    
    async def _pre_execute(self):
        await self._await_signal('execute')
        await self._update_config()
        await self._execute()
        self._main_task = self._task_manager.create_task(self._pre_execute())
    
    @abstractmethod
    async def _execute(self):
        raise NotImplementedError()

In [ ]:
class MyExecComponent(ExecComponent):
    async def _execute(self):
        print("I'm executing!")
    
comp_process = MyExecComponent()
await comp_process.start()
await asyncio.sleep(0)
await comp_process.terminate()

I'm executing!
